이번 Lab을 통해 XOR 문제를 다층 퍼셉트론으로 구현해보겠습니다!

In [ ]:
import torch
import torch.nn as nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

XOR 문제를 풀기 위한 입력과 출력을 정의해줍니다.

In [ ]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

입력층, 은닉층 3개, 출력층을 가지는 인공신경망을 구현해보겠습니다.

In [ ]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input layer의 크기를 생각해보세요.
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # output layer의 크기를 생각해보세요.
          nn.Sigmoid()
          ).to(device)

In [ ]:
"""
# 이렇게 구현해도 됩니다.
# nn layers
linear1 = torch.nn.Linear(2, 10, bias=True) # input layer의 크기를 생각해보세요.
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True) # output layer의 크기를 생각해보세요.
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)
"""

이제 비용 함수와 옵타마이저를 선언합니다.

In [ ]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device) # 이진 분류에서 사용하는 크로스엔트로피 함수를 사용하세요.
optimizer = torch.optim.SGD(model.parameters(), lr=1) # SGD optimizer를 사용하고 learing rate는 1로 설정하세요.

총 10,001번의 에포크를 수행합니다. 각 에포크마다 역전파가 수행된다고 보면 되겠습니다.

In [ ]:
for step in range(10001):
    optimizer.zero_grad() # gradient를 0으로 초기화
    hypothesis = model(X) # 모델을 생성해서 hypothesis를 구해보세요.

    # cost/loss function
    cost = criterion(hypothesis, Y)# 위에서 정의한 비용함수를 사용하세요.
    cost.backward() # 비용 함수를 미분하여 gradient 계산
    optimizer.step()  # W와 b를 업데이트

    if step % 100 == 0:
        print(step, cost.item())

0 0.7408947944641113
100 0.6931389570236206
200 0.6931374669075012
300 0.6931358575820923
400 0.6931341886520386
500 0.6931324005126953
600 0.693130612373352
700 0.6931287050247192
800 0.6931267380714417
900 0.6931245923042297
1000 0.6931224465370178
1100 0.693120002746582
1200 0.6931175589561462
1300 0.6931147575378418
1400 0.693111777305603
1500 0.6931086778640747
1600 0.6931051015853882
1700 0.6931012868881226
1800 0.6930969953536987
1900 0.6930922865867615
2000 0.6930870413780212
2100 0.693081259727478
2200 0.693074643611908
2300 0.6930670738220215
2400 0.6930585503578186
2500 0.6930487155914307
2600 0.6930371522903442
2700 0.6930238008499146
2800 0.6930078268051147
2900 0.6929886937141418
3000 0.6929655075073242
3100 0.6929367780685425
3200 0.6929006576538086
3300 0.6928543448448181
3400 0.6927932500839233
3500 0.692710280418396
3600 0.692592978477478
3700 0.6924187541007996
3800 0.6921423673629761
3900 0.6916618347167969
4000 0.6907041072845459
4100 0.6883158683776855
4200 0.6790

이제 모델이 XOR 문제를 풀 수 있는지 테스트 해봅시다.

In [ ]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[6.8849222e-05]
 [9.9988163e-01]
 [9.9989247e-01]
 [1.5689511e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


실제값은 0, 1, 1, 0이며 예측값은 0, 1, 1, 0으로 문제를 해결하는 모습을 보여줍니다.

Q1. Backpropagation이란?

=> Backpropagation(역전파)은 목표값과 실제 모델이 계산한 값이 얼마나 차이가 나는지 구한 후 그 오차값을 다시 뒤로 전파해가면서 각 노드가 갖고 있는 변수들을 갱신하는 알고리즘이다. 

Q2. Backpropagation은 어떤 방식을 이용해 weight, bias 같은 변수들을 업데이트 할까요?

=> 경사하강법을 이용해 오차가 최소화 되는 방향으로 진행, 복잡한 미분은 연쇄법칙을 이용한다.